# Naive Bayes Spell Correction

This notebook demonstrates a probabilistic approach to spell correction using the **Naive Bayes** rule.

## Theory
We want to find the correction $c$ that is most likely given the observed typo $w$ (from OCR).

$$ \hat{c} = \arg\max_{c} P(c | w) $$

Using Bayes' Theorem:

$$ P(c | w) = \frac{P(w | c) P(c)}{P(w)} $$

Since $P(w)$ is constant for all candidates $c$, we maximize:

$$ \hat{c} = \arg\max_{c} P(w | c) P(c) $$

Where:
- **$P(c)$ (The Prior)**: The probability that word $c$ appears in the language (Word Frequency).
- **$P(w | c)$ (The Likelihood)**: The probability that the user/OCR types $w$ when they meant $c$ (Error Model / Edit Distance).

In [36]:
import sys
import os
from collections import Counter
import numpy as np

# Setup paths
sys.path.append(os.path.abspath('..'))
DATA_PATH = '../data/urdu_words.txt'

## 1. The Language Model $P(c)$
We estimate this using word counts from our corpus.

In [37]:
class LanguageModel:
    def __init__(self, path):
        self.words = Counter()
        self.N = 0
        self.train(path)
        
    def train(self, path):
        if not os.path.exists(path):
            print("Dataset not found!")
            return
        with open(path, 'r', encoding='utf-8') as f:
            text = f.read()
            tokens = text.split()
            self.words.update(tokens)
            self.N = sum(self.words.values())
            
    def P(self, word):
        "Probability of word: Count(word) / Total_Words"
        return self.words[word] / self.N

lm = LanguageModel(DATA_PATH)
print(f"Model loaded with {len(lm.words)} unique words.")

Model loaded with 154781 unique words.


## 2. The Error Model $P(w | c)$
This estimates the chance of an error. We can model this using **Edit Distance**.
- Distance 0 (same word): Very high probability (e.g., 0.95)
- Distance 1: Lower probability (e.g., 0.04)
- Distance 2: Very low probability (e.g., 0.009)

Ideally, you would use a 'Confusion Matrix' of character errors (e.g., how often 'p' is OCR'd as 'q'), but for now, we use a simple exponential decay based on Edit Distance.

In [38]:
def edit_distance(s1, s2):
    if len(s1) < len(s2):
        return edit_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def error_probability(typo, candidate, dist=None):
    if dist is None:
        dist = edit_distance(typo, candidate)
    
    # Simple heuristic model
    if dist == 0:
        return 0.90
    elif dist == 1:
        return 0.09
    elif dist == 2:
        return 0.01
    else:
        return 1e-10

## 3. The Classifier
We generate candidates (valid words within edit dist 1 or 2) and score them.

In [39]:
def generate_candidates(word, vocabulary):
    # For efficiency, in a real system we wouldn't scan the whole vocab.
    # We would use the 'edits1' and 'edits2' generator approach.
    # Here, for demonstration, let's assume we have a way to get 'potential' candidates.
    # Let's pretend our generator gave us these:
    
    # (In a real scenario, use the method from src/read.py to generate close words)
    return vocabulary # Placeholder: Real Naive Bayes iterates generated edits, not full vocab

def solve(typo, candidates):
    scores = {}
    for c in candidates:
        prior = lm.P(c)
        if prior == 0: continue # Ignore unknown words
        
        dist = edit_distance(typo, c)
        likelihood = error_probability(typo, c, dist)
        
        # P(c|w) ~ P(w|c) * P(c)
        scores[c] = likelihood * prior
        
    # Return best
    if not scores:
        return typo, {}
    return max(scores, key=scores.get), scores


## 4. Demonstration
Let's test with 'کتاپ' (Typo for 'کتاب').

In [40]:
typo = 'کتاپ'

# Efficient Candidate Generation (Using Peter Norvig's edits1 logic for demo)
def edits1(word):
    letters = 'ابپتٹثجچحخدڈذرڑزژسشصضطظعغفقکگلمنںوہیے'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# Generate candidates that are actually in our dictionary
candidates = set(e for e in edits1(typo) if e in lm.words)

print(f"Candidates for {typo}: {candidates}")

best, all_scores = solve(typo, candidates)

print(f"\nBest Correction: {best}")

print("\nDetailed Scores (Likelihood * Prior):")
for c in candidates:
    print(f"{c}: {all_scores[c]:.2e} (Prior: {lm.P(c):.2e}, Dist: {edit_distance(typo, c)})")

Candidates for کتاپ: {'کتان', 'کتا', 'کتاب', 'تاپ'}

Best Correction: کتان

Detailed Scores (Likelihood * Prior):
کتان: 5.81e-07 (Prior: 6.46e-06, Dist: 1)
کتا: 5.81e-07 (Prior: 6.46e-06, Dist: 1)
کتاب: 5.81e-07 (Prior: 6.46e-06, Dist: 1)
تاپ: 5.81e-07 (Prior: 6.46e-06, Dist: 1)


## 5. Evaluation on Synthetic Data
We evaluate the accuracy on 500 randomly generated typos, similar to the KNN evaluation.

In [41]:
import random
import time

def generate_typo(word):
    if len(word) < 2: return word
    urdu_chars = 'ابپتٹثجچحخدڈذرڑزژسشصضطظعغفقکگلمنںوہیے'
    op = random.choice(['insert', 'delete', 'replace', 'transpose'])
    word = list(word)
    idx = random.randint(0, len(word) - 1)
    
    if op == 'insert': word.insert(idx, random.choice(urdu_chars))
    elif op == 'delete': word.pop(idx)
    elif op == 'replace': word[idx] = random.choice(urdu_chars)
    elif op == 'transpose' and idx < len(word)-1: word[idx], word[idx+1] = word[idx+1], word[idx]
        
    return "".join(word)

# Sample 500 words
random.seed(42)
valid_words = [w for w in lm.words if len(w) > 3]
test_set = [(generate_typo(w), w) for w in random.sample(valid_words, 500)]

print(f"Generated {len(test_set)} pairs.")

def get_candidates(typo):
    c1 = set(e for e in edits1(typo) if e in lm.words)
    return c1 if c1 else {typo} 

correct_count = 0
start_time = time.time()

print("Running Evaluation...")
for typo, truth in test_set:
    candidates = get_candidates(typo)
    best_pred, _ = solve(typo, candidates)
    
    if best_pred == truth:
        correct_count += 1

duration = time.time() - start_time
accuracy = (correct_count / len(test_set)) * 100
print(f"Accuracy: {accuracy:.2f}%")
print(f"Time: {duration:.2f}s")

Generated 500 pairs.
Running Evaluation...
Accuracy: 68.20%
Time: 0.11s
